In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import scipy.io
import torch
# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [3]:
data = scipy.io.loadmat("data/lda/lda_on_grid.mat")
data = data["lda_on_grid"]
X = torch.tensor(data[:, :3])
X[:, 0] = 2.0 * X[:, 0] - 1.0
X[:, 1] = torch.log2(X[:, 1]) / 10.0
X[:, 2] = torch.log2(X[:, 2]) / 14.0
objective_X = -torch.tensor(data[:, 3]).unsqueeze(-1)
cost_X = torch.tensor(data[:, 4]).unsqueeze(-1) / 3600.0

In [4]:
dim = 3
maximize = True

In [5]:
from pandora_bayesopt.utils import fit_gp_model
objective_model = fit_gp_model(X, objective_X, input_standardize=True)

In [6]:
def objective_function(X):
    if X.ndim == 1:
        X = X.unsqueeze(0)
    posterior_X = objective_model.posterior(X)
    objective_X = posterior_X.mean.detach()
    return 0.001*objective_X

In [7]:
objective_model.eval()

SingleTaskGP(
  (likelihood): FixedNoiseGaussianLikelihood(
    (noise_covar): FixedGaussianNoise()
  )
  (mean_module): ConstantMean()
  (covar_module): ScaleKernel(
    (base_kernel): MaternKernel(
      (lengthscale_prior): GammaPrior()
      (raw_lengthscale_constraint): Positive()
    )
    (outputscale_prior): GammaPrior()
    (raw_outputscale_constraint): Positive()
  )
  (outcome_transform): Standardize()
)

In [8]:
log_cost_X = torch.log(cost_X)
log_cost_model = fit_gp_model(X, log_cost_X, input_standardize=True)

In [9]:
def cost_function(X):
    posterior_X = log_cost_model.posterior(X)
    cost_X = torch.exp(posterior_X.mean)
    return cost_X

In [10]:
bounds = torch.stack([torch.zeros(dim), torch.ones(dim)])
budget = 5
seed = 12

In [11]:
from botorch.acquisition import ExpectedImprovement
from pandora_bayesopt.acquisition.gittins import GittinsIndex
from pandora_bayesopt.acquisition.ei_puc import ExpectedImprovementWithCost
from pandora_bayesopt.bayesianoptimizer import BayesianOptimizer
from botorch.utils.sampling import draw_sobol_samples

In [12]:
global_optimum_value = -1.2606842790227435

In [13]:
torch.manual_seed(seed)
init_x = draw_sobol_samples(bounds=bounds, n=1, q=2*(dim+1)).squeeze(0)

# Example usage with EI
EI_optimizer = BayesianOptimizer(objective=objective_function, dim=dim, maximize=maximize, cost=cost_function, initial_points=init_x, input_standardize=True)
EI_optimizer.run_until_budget(budget=budget, acquisition_function_class=ExpectedImprovement)
EI_best_history = EI_optimizer.get_best_history()
EI_regret_history = EI_optimizer.get_regret_history(global_optimum_value)
EI_cost_history = EI_optimizer.get_cost_history()

Iteration 0, New point: [0.57766536 0.57438965 0.75741616], New value: -1.2984029632645637
Best observed value: -1.2704712372093203
Current acquisition value: tensor(0.0894)
Cumulative cost: 4.182386782394579

Iteration 1, New point: [0.20091817 0.29102848 0.87349667], New value: -1.2885565059815858
Best observed value: -1.2704712372093203
Current acquisition value: tensor(0.0570)
Cumulative cost: 7.379394895636061



In [14]:
from pandora_bayesopt.acquisition.budgeted_multi_step_ei import BudgetedMultiStepLookaheadEI

torch.manual_seed(seed)

init_x = torch.rand(1, dim)

# Example usage with BMSEI
BMSEI_optimizer = BayesianOptimizer(objective=objective_function, dim=dim, maximize=maximize, cost=cost_function, initial_points=init_x, input_standardize=True)
BMSEI_optimizer.run_until_budget(budget=budget, acquisition_function_class=BudgetedMultiStepLookaheadEI)
BMSEI_best_history = BMSEI_optimizer.get_best_history()
BMSEI_regret_history = BMSEI_optimizer.get_regret_history(global_optimum_value)
BMSEI_cost_history = BMSEI_optimizer.get_cost_history()

11.102708568121024


/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/transforms/outcome.py:304: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  stdvs = Y.std(dim=-2, keepdim=True)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraBayesOPT/pandorabayesopt_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1760.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)


AssertionError: Expected the output shape to match either the t-batch shape of X, or the `model.batch_shape` in the case of acquisition functions using batch models; but got output with shape torch.Size([5, 5]) for X with shape torch.Size([5, 1, 3]).